In [1]:
from bs4 import BeautifulSoup

import pandas as pd
import requests
import re

pd.set_option("display.max_colwidth", 9999)
# pd.set_option("display.max_rows", 9999)

In [2]:
raw_recipes = pd.read_csv("food recipes dataset\RAW_recipes.csv")
raw_recipes.head(1)

,Unnamed: 0,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course', 'main-ingredient', 'cuisine', 'preparation', 'occasion', 'north-american', 'side-dishes', 'vegetables', 'mexican', 'easy', 'fall', 'holiday-event', 'vegetarian', 'winter', 'dietary', 'christmas', 'seasonal', 'squash']","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'depending on size of squash , cut into half or fourths', 'remove seeds', 'for spicy squash , drizzle olive oil or melted butter over each cut squash piece', 'season with mexican seasoning mix ii', 'for sweet squash , drizzle melted honey , butter , grated piloncillo over each cut squash piece', 'season with sweet mexican spice mix', 'bake at 350 degrees , again depending on size , for 40 minutes up to an hour , until a fork can easily pierce the skin', 'be careful not to burn the squash especially if you opt to use sugar or butter', 'if you feel more comfortable , cover the squash with aluminum foil the first half hour , give or take , of baking', 'if desired , season with salt']","autumn is my favorite time of year to cook! this recipe \r\ncan be prepared either spicy or sweet, your choice!\r\ntwo of my posted mexican-inspired seasoning mix recipes are offered as suggestions.","['winter squash', 'mexican seasoning', 'mixed spice', 'honey', 'butter', 'olive oil', 'salt']",7


In [3]:
final_scraped_data = pd.DataFrame({"Recipe Name": [], "Steps": [], "Ingredients": []})

for i in range(0, 200):

    # recipe name
    recipe_name = re.sub(r"\s+", " ", raw_recipes.iloc[i]["name"])

    # replace one or more spaces with '-' and then append the id to create unique id url
    unique_id_recipe_url = (
        re.sub(r"\s+", "-", raw_recipes.iloc[i]["name"])
        + "-"
        + str(raw_recipes.iloc[i]["id"])
    )
    url = ("https://www.food.com/recipe/" + unique_id_recipe_url).strip()
    print(str(i)+". "+url)

    # create request and get content
    raw = requests.get(url)
    soup = BeautifulSoup(raw.content, "html.parser")

    # get the steps of recipe
    steps_list = []
    steps = soup.find("ul", attrs={"class": "recipe-directions__steps"})

    for step in steps.find_all("li"):

        # if recipe direction contains the word "note" we will ignore that step
        if not re.search("note", step.text, flags=re.IGNORECASE):
            steps_list.append(step.text.strip())

    ##########################################################################################

    # get the ingredients and quantity of recipe
    # current key used to keep track of subingredients
    current_key = "main_ingredients"
    ingredient_dictionary = {current_key: []}

    # get the ingredients unordered list from website
    ingredients = soup.find("ul", attrs={"class": "recipe-ingredients__list"})

    for ingredient in ingredients.find_all("li"):

        if (
            ingredient.find(
                "div", attrs={"class": "recipe-ingredients__ingredient-heading"}
            )
            is not None
        ):
            current_key = ingredient.find(
                "div", attrs={"class": "recipe-ingredients__ingredient-heading"}
            ).text.strip()
            ingredient_dictionary[current_key] = []
            # since if header present then no ingredient and quantity  present
            continue

        # if quantity not specified then use N/A
        quantity_scraped = re.sub(
            "\s+",
            "",
            ingredient.find(
                "div", attrs={"class": "recipe-ingredients__ingredient-quantity"}
            ).text.strip(),
        )
        quantity = "N/A" if quantity_scraped == "" else quantity_scraped

        ingredient_scraped = ingredient.find(
            "div", attrs={"class": "recipe-ingredients__ingredient-parts"}
        ).text.strip()

        hyperlink_data = []
        for hyper_link_scraped in ingredient.find(
            "div", attrs={"class": "recipe-ingredients__ingredient-parts"}
        ).find_all("a"):
            hyperlink_data.append(hyper_link_scraped.text)

        hyperlink_data_cleaned = (
            "NA" if ",".join(hyperlink_data) == "" else ",".join(hyperlink_data)
        )

        ingredient_part = re.sub("\s+", " ", ingredient_scraped)
        ingredient_dictionary[current_key].append(
            [quantity, ingredient_part, hyperlink_data_cleaned]
        )

    # append all the scraped data into pandas dataframe
    final_scraped_data = final_scraped_data.append(
        {
            "Recipe Name": recipe_name,
            "Steps": steps_list,
            "Ingredients": ingredient_dictionary,
        },
        ignore_index=True,
    )

0. https://www.food.com/recipe/arriba-baked-winter-squash-mexican-style-137739
1. https://www.food.com/recipe/a-bit-different-breakfast-pizza-31490
2. https://www.food.com/recipe/all-in-the-kitchen-chili-112140
3. https://www.food.com/recipe/alouette-potatoes-59389
4. https://www.food.com/recipe/amish-tomato-ketchup-for-canning-44061
5. https://www.food.com/recipe/apple-a-day-milk-shake-5289
6. https://www.food.com/recipe/aww-marinated-olives-25274
7. https://www.food.com/recipe/backyard-style-barbecued-ribs-67888
8. https://www.food.com/recipe/bananas-4-ice-cream-pie-70971
9. https://www.food.com/recipe/beat-this-banana-bread-75452
10. https://www.food.com/recipe/berry-good-sandwich-spread-109439
11. https://www.food.com/recipe/better-than-sex-strawberries-42198
12. https://www.food.com/recipe/better-then-bush-s-baked-beans-67547
13. https://www.food.com/recipe/boat-house-collard-greens-107517
14. https://www.food.com/recipe/calm-your-nerves-tonic-39959
15. https://www.food.com/recipe

125. https://www.food.com/recipe/2000-meatloaf-475785
126. https://www.food.com/recipe/25-pumpkin-pie-17265
127. https://www.food.com/recipe/25-000-casserole-50662
128. https://www.food.com/recipe/250-chocolate-chip-cookies-recipe-177187
129. https://www.food.com/recipe/250-00-chocolate-chip-cookies-118843
130. https://www.food.com/recipe/300-icing-208179
131. https://www.food.com/recipe/5-tacos-500166
132. https://www.food.com/recipe/50-chili-for-the-crockpot-501028
133. https://www.food.com/recipe/500-margarita-bread-136602
134. https://www.food.com/recipe/ara-s-potato-oup-69190
135. https://www.food.com/recipe/blepandekager-danish-apple-pancakes-503475
136. https://www.food.com/recipe/lplermagrone-522861
137. https://www.food.com/recipe/lplermagrone-herdsman-s-macaroni-457136
138. https://www.food.com/recipe/lplermagronen-455351
139. https://www.food.com/recipe/rter-med-flsk-pea-soup-with-pork-333797
140. https://www.food.com/recipe/rtsoppa-swedish-yellow-pea-soup-428056
141. https:

In [4]:
len(final_scraped_data)

200

In [5]:
final_scraped_data.head(10)
# final_scraped_data

,Recipe Name,Steps,Ingredients
0,arriba baked winter squash mexican style,"[Depending on size of squash, cut into half or fourths., Remove seeds., For spicy squash, drizzle (roughly 1/2 teaspoon) olive oil or melted butter over each cut squash piece. Season with Mexican Seasoning Mix II., For sweet squash, drizzle (roughly 1/2 teaspoon) melted honey, butter, grated piloncillo (or any combination of) over each cut squash piece. Season with Sweet Mexican Spice Mix., Bake at 350 degrees, again depending on size, for 40 minutes up to an hour, until a fork can easily pierce the skin., Be careful not to burn the squash especially if you opt to use sugar or butter. If you feel more comfortable, cover the squash with aluminum foil the first half hour, give or take, of baking., If desired, season with salt.]","{'main_ingredients': [['1', 'lb winter squash (such as hubbard, acorn, delicata, spaghetti, etc.)', 'NA'], ['1-2', 'teaspoon Mexican seasoning, mix to taste (see my Mexican Seasoning Mix II (Salt Free))', 'Mexican Seasoning Mix II (Salt Free)'], ['1-2', 'teaspoon mexican mixed spice, to taste (see my Sweet Mexican Spice Seasoning Mix)', 'mixed spice,Sweet Mexican Spice Seasoning Mix'], ['N/A', 'honey (or piloncillo)', 'honey'], ['N/A', 'butter', 'butter'], ['N/A', 'olive oil', 'olive oil'], ['N/A', 'salt', 'salt']]}"
1,a bit different breakfast pizza,"[Preheat oven to 425 degrees F., Press dough into the bottom and sides of a 12 inch pizza pan., Bake for 5 minutes until set but not browned., Cut sausage into small pieces., Whisk eggs and milk in a bowl until frothy., Spoon sausage over baked crust and sprinkle with cheese., Pour egg mixture slowly over sausage and cheese., S& P to taste., Bake 15-20 minutes or until eggs are set and crust is brown.]","{'main_ingredients': [['1', '(10 ounce) can prepared pizza crust (or use your favorite recipe)', 'NA'], ['1', '(7 ounce) package prebrowned sausage patty, thawed', 'NA'], ['3', 'eggs', 'eggs'], ['1⁄2', 'cup milk', 'milk'], ['N/A', 'salt and pepper', 'NA'], ['2', 'cups shredded cheese (pizza-style or use your favorite)', 'cheese']]}"
2,all in the kitchen chili,"[Brown ground beef in large pot., Add chopped onions to ground beef when almost brown and sautee until wilted., Add all other ingredients., Add kidney beans if you like beans in your chili., Cook in slow cooker on high for 2-3 hours or 6-8 hours on low., Serve with cold clean lettuce and shredded cheese (It's a Cajun thing).]","{'main_ingredients': [['2', 'lbs ground beef', 'ground beef'], ['11⁄2', 'medium yellow onions', 'yellow onions'], ['1', '(15 ounce) can diced tomatoes', 'diced tomatoes'], ['1', '(6 ounce) can tomato paste', 'tomato paste'], ['1', '(15 ounce) can tomato soup', 'NA'], ['1', '(11 ounce) can Rotel Tomatoes', 'Rotel Tomatoes'], ['2', '(15 ounce) cans kidney beans (optional)', 'kidney beans'], ['1', 'cup water', 'water'], ['41⁄2', 'tablespoons chili powder', 'chili powder'], ['2', 'teaspoons ground cumin', 'ground cumin'], ['2', 'teaspoons salt', 'salt'], ['N/A', 'lettuce', 'lettuce'], ['N/A', 'shredded cheddar cheese', 'cheddar cheese']]}"
3,alouette potatoes,"[Place potatoes in a large pot of lightly salted water and bring to a gentle boil., Cook until potatoes are just tender., Drain., Place potatoes in a large bowl and add all ingredients except the""Alouette""., Mix well and transfer to a buttered 8x8 inch glass baking dish with 2 inch sides., Press the potatoes with a spatula to make top as flat as possible., Set aside for 2 hours at room temperature., Preheat oven to 350^F., Spread""Alouette"" evenly over potatoes and bake 15 minutes., Divide between plates., Garnish with finely diced red and yellow bell peppers.]","{'main_ingredients': [['2', '(4 ounce) packages alouette spreadable cheese with garlic and herbs', 'NA'], ['12', 'large new potatoes, sliced 1/4 inch thick', 'new potatoes'], ['2', 'shallots, diced finely', 'shallots'], ['2', 'teaspoons chopped parsley', 'parsley'], ['1', 'teaspoon chopped

In [6]:
# storing as pickle to preserve the data structure in each column
final_scraped_data.to_pickle("./scraped_recipes")

In [7]:
# scraped data
for s in final_scraped_data["Steps"][:10]:
    for i, x in enumerate(s):
        print(str(i + 1) + ". " + str(x))
    print("-------------")

1. Depending on size of squash, cut into half or fourths.
2. Remove seeds.
3. For spicy squash, drizzle (roughly 1/2 teaspoon) olive oil or melted butter over each cut squash piece. Season with Mexican Seasoning Mix II.
4. For sweet squash, drizzle (roughly 1/2 teaspoon) melted honey, butter, grated piloncillo (or any combination of) over each cut squash piece. Season with Sweet Mexican Spice Mix.
5. Bake at 350 degrees, again depending on size, for 40 minutes up to an hour, until a fork can easily pierce the skin.
6. Be careful not to burn the squash especially if you opt to use sugar or butter. If you feel more comfortable, cover the squash with aluminum foil the first half hour, give or take, of baking.
7. If desired, season with salt.
-------------
1. Preheat oven to 425 degrees F.
2. Press dough into the bottom and sides of a 12 inch pizza pan.
3. Bake for 5 minutes until set but not browned.
4. Cut sausage into small pieces.
5. Whisk eggs and milk in a bowl until frothy.
6. Spoon

In [8]:
# scraped data
for s in final_scraped_data["Ingredients"][:10]:
    for k, v in s.items():
        print(k, v)
        print()
    print("-------------")

main_ingredients [['1', 'lb winter squash (such as hubbard, acorn, delicata, spaghetti, etc.)', 'NA'], ['1-2', 'teaspoon Mexican seasoning, mix to taste (see my Mexican Seasoning Mix II (Salt Free))', 'Mexican Seasoning Mix II (Salt Free)'], ['1-2', 'teaspoon mexican mixed spice, to taste (see my Sweet Mexican Spice Seasoning Mix)', 'mixed spice,Sweet Mexican Spice Seasoning Mix'], ['N/A', 'honey (or piloncillo)', 'honey'], ['N/A', 'butter', 'butter'], ['N/A', 'olive oil', 'olive oil'], ['N/A', 'salt', 'salt']]

-------------
main_ingredients [['1', '(10 ounce) can prepared pizza crust (or use your favorite recipe)', 'NA'], ['1', '(7 ounce) package prebrowned sausage patty, thawed', 'NA'], ['3', 'eggs', 'eggs'], ['1⁄2', 'cup milk', 'milk'], ['N/A', 'salt and pepper', 'NA'], ['2', 'cups shredded cheese (pizza-style or use your favorite)', 'cheese']]

-------------
main_ingredients [['2', 'lbs ground beef', 'ground beef'], ['11⁄2', 'medium yellow onions', 'yellow onions'], ['1', '(15 oun

In [9]:
verify = pd.read_pickle("./scraped_recipes")

In [10]:
verify.head()

,Recipe Name,Steps,Ingredients
0,arriba baked winter squash mexican style,"[Depending on size of squash, cut into half or fourths., Remove seeds., For spicy squash, drizzle (roughly 1/2 teaspoon) olive oil or melted butter over each cut squash piece. Season with Mexican Seasoning Mix II., For sweet squash, drizzle (roughly 1/2 teaspoon) melted honey, butter, grated piloncillo (or any combination of) over each cut squash piece. Season with Sweet Mexican Spice Mix., Bake at 350 degrees, again depending on size, for 40 minutes up to an hour, until a fork can easily pierce the skin., Be careful not to burn the squash especially if you opt to use sugar or butter. If you feel more comfortable, cover the squash with aluminum foil the first half hour, give or take, of baking., If desired, season with salt.]","{'main_ingredients': [['1', 'lb winter squash (such as hubbard, acorn, delicata, spaghetti, etc.)', 'NA'], ['1-2', 'teaspoon Mexican seasoning, mix to taste (see my Mexican Seasoning Mix II (Salt Free))', 'Mexican Seasoning Mix II (Salt Free)'], ['1-2', 'teaspoon mexican mixed spice, to taste (see my Sweet Mexican Spice Seasoning Mix)', 'mixed spice,Sweet Mexican Spice Seasoning Mix'], ['N/A', 'honey (or piloncillo)', 'honey'], ['N/A', 'butter', 'butter'], ['N/A', 'olive oil', 'olive oil'], ['N/A', 'salt', 'salt']]}"
1,a bit different breakfast pizza,"[Preheat oven to 425 degrees F., Press dough into the bottom and sides of a 12 inch pizza pan., Bake for 5 minutes until set but not browned., Cut sausage into small pieces., Whisk eggs and milk in a bowl until frothy., Spoon sausage over baked crust and sprinkle with cheese., Pour egg mixture slowly over sausage and cheese., S& P to taste., Bake 15-20 minutes or until eggs are set and crust is brown.]","{'main_ingredients': [['1', '(10 ounce) can prepared pizza crust (or use your favorite recipe)', 'NA'], ['1', '(7 ounce) package prebrowned sausage patty, thawed', 'NA'], ['3', 'eggs', 'eggs'], ['1⁄2', 'cup milk', 'milk'], ['N/A', 'salt and pepper', 'NA'], ['2', 'cups shredded cheese (pizza-style or use your favorite)', 'cheese']]}"
2,all in the kitchen chili,"[Brown ground beef in large pot., Add chopped onions to ground beef when almost brown and sautee until wilted., Add all other ingredients., Add kidney beans if you like beans in your chili., Cook in slow cooker on high for 2-3 hours or 6-8 hours on low., Serve with cold clean lettuce and shredded cheese (It's a Cajun thing).]","{'main_ingredients': [['2', 'lbs ground beef', 'ground beef'], ['11⁄2', 'medium yellow onions', 'yellow onions'], ['1', '(15 ounce) can diced tomatoes', 'diced tomatoes'], ['1', '(6 ounce) can tomato paste', 'tomato paste'], ['1', '(15 ounce) can tomato soup', 'NA'], ['1', '(11 ounce) can Rotel Tomatoes', 'Rotel Tomatoes'], ['2', '(15 ounce) cans kidney beans (optional)', 'kidney beans'], ['1', 'cup water', 'water'], ['41⁄2', 'tablespoons chili powder', 'chili powder'], ['2', 'teaspoons ground cumin', 'ground cumin'], ['2', 'teaspoons salt', 'salt'], ['N/A', 'lettuce', 'lettuce'], ['N/A', 'shredded cheddar cheese', 'cheddar cheese']]}"
3,alouette potatoes,"[Place potatoes in a large pot of lightly salted water and bring to a gentle boil., Cook until potatoes are just tender., Drain., Place potatoes in a large bowl and add all ingredients except the""Alouette""., Mix well and transfer to a buttered 8x8 inch glass baking dish with 2 inch sides., Press the potatoes with a spatula to make top as flat as possible., Set aside for 2 hours at room temperature., Preheat oven to 350^F., Spread""Alouette"" evenly over potatoes and bake 15 minutes., Divide between plates., Garnish with finely diced red and yellow bell peppers.]","{'main_ingredients': [['2', '(4 ounce) packages alouette spreadable cheese with garlic and herbs', 'NA'], ['12', 'large new potatoes, sliced 1/4 inch thick', 'new potatoes'], ['2', 'shallots, diced finely', 'shallots'], ['2', 'teaspoons chopped parsley', 'parsley'], ['1', 'teaspoon chopped

In [11]:
# pickled data
for s in verify["Steps"][:10]:
    for i, x in enumerate(s):
        print(str(i + 1) + ". " + str(x))
    print("-------------")

1. Depending on size of squash, cut into half or fourths.
2. Remove seeds.
3. For spicy squash, drizzle (roughly 1/2 teaspoon) olive oil or melted butter over each cut squash piece. Season with Mexican Seasoning Mix II.
4. For sweet squash, drizzle (roughly 1/2 teaspoon) melted honey, butter, grated piloncillo (or any combination of) over each cut squash piece. Season with Sweet Mexican Spice Mix.
5. Bake at 350 degrees, again depending on size, for 40 minutes up to an hour, until a fork can easily pierce the skin.
6. Be careful not to burn the squash especially if you opt to use sugar or butter. If you feel more comfortable, cover the squash with aluminum foil the first half hour, give or take, of baking.
7. If desired, season with salt.
-------------
1. Preheat oven to 425 degrees F.
2. Press dough into the bottom and sides of a 12 inch pizza pan.
3. Bake for 5 minutes until set but not browned.
4. Cut sausage into small pieces.
5. Whisk eggs and milk in a bowl until frothy.
6. Spoon

In [12]:
# pickled data
for s in verify["Ingredients"][:10]:
    for k, v in s.items():
        print(k, v)
        print()
    print("-------------")

main_ingredients [['1', 'lb winter squash (such as hubbard, acorn, delicata, spaghetti, etc.)', 'NA'], ['1-2', 'teaspoon Mexican seasoning, mix to taste (see my Mexican Seasoning Mix II (Salt Free))', 'Mexican Seasoning Mix II (Salt Free)'], ['1-2', 'teaspoon mexican mixed spice, to taste (see my Sweet Mexican Spice Seasoning Mix)', 'mixed spice,Sweet Mexican Spice Seasoning Mix'], ['N/A', 'honey (or piloncillo)', 'honey'], ['N/A', 'butter', 'butter'], ['N/A', 'olive oil', 'olive oil'], ['N/A', 'salt', 'salt']]

-------------
main_ingredients [['1', '(10 ounce) can prepared pizza crust (or use your favorite recipe)', 'NA'], ['1', '(7 ounce) package prebrowned sausage patty, thawed', 'NA'], ['3', 'eggs', 'eggs'], ['1⁄2', 'cup milk', 'milk'], ['N/A', 'salt and pepper', 'NA'], ['2', 'cups shredded cheese (pizza-style or use your favorite)', 'cheese']]

-------------
main_ingredients [['2', 'lbs ground beef', 'ground beef'], ['11⁄2', 'medium yellow onions', 'yellow onions'], ['1', '(15 oun